In [5]:
import re
import pandas as pd
import os

def format_text_to_csv(input_file, output_csv, seiteinen, jichitai, kubun):
    """
    テキストファイルを読み込み、「号」レベル（(1), (2), （1）など）の前の改行を削除する。
    整形したテキストをCSVファイルの「本文」カラムに追加する。
    
    Parameters:
    - input_file: 入力テキストファイルのパス
    - output_csv: 出力CSVファイルのパス
    - seiteinen: 制定年
    - jichitai: 自治体名
    - kubun: 区分
    """
    with open(input_file, 'r', encoding='utf-8') as f:
        text = f.read()
    
    # 行ごとに分割
    lines = text.split('\n')
    
    # 整形後のテキストを格納
    formatted_lines = []
    
    for i, line in enumerate(lines):
        line_stripped = line.strip()
        
        # 空行はスキップ
        if not line_stripped:
            continue
        
        # 「号」パターン: (1), (2), （1）, （2） など（括弧+数字）で始まる行
        is_gou = re.match(r'^[（\(]\d+[）\)]', line_stripped)
        
        if is_gou and formatted_lines:
            # 号の場合は改行せず、直前の行に連結
            formatted_lines[-1] += line_stripped
        else:
            # それ以外（条、項、ア、イなど）は改行を保持
            formatted_lines.append(line_stripped)
    
    # 整形されたテキストを結合
    formatted_text = '\n'.join(formatted_lines)
    
    # 新しい行のデータを作成
    new_row = {
        '本文': formatted_text,
        '制定年': seiteinen,
        '自治体': jichitai,
        '区分': kubun
    }
    
    # CSVファイルが存在する場合は読み込み、存在しない場合は新規作成
    if os.path.exists(output_csv):
        df = pd.read_csv(output_csv)
        # 新しい行を追加
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    else:
        # 新規作成
        df = pd.DataFrame([new_row])
    
    # CSVに保存
    df.to_csv(output_csv, index=False, encoding='utf-8')
    
    print(f"整形完了: {output_csv} に新しい行を追加しました")
    print(f"総行数: {len(df)}")

# 使用例
input_file = 'input.txt'  # 入力ファイル名を指定
output_csv = 'main2.0.csv'  # 出力CSVファイル名を指定

# 各カラムの値を指定
seiteinen = '2025'  # 制定年
jichitai = '大空町'  # 自治体名
kubun = '条例'  # 区分

format_text_to_csv(input_file, output_csv, seiteinen, jichitai, kubun)

整形完了: main2.0.csv に新しい行を追加しました
総行数: 1


# テキスト整形ツール（CSV出力版）

このノートブックは、条例などの法律文書を整形し、CSVファイルに追記するためのツールです。

**機能:**
- 「号」レベル（`(1)`, `(2)`, `（1）` など）の前の改行を削除
- それ以外の階層（`第○条`、`2`、`ア` など）の前では改行を保持
- 整形したテキストを `main2.0.csv` の「本文」カラムに追加
- 実行するたびに新しい行が追加される

**使い方:**
1. 入力ファイル（`input.txt`）を準備
2. 下のセルで以下のパラメータを指定:
   - `input_file`: 入力ファイル名
   - `output_csv`: 出力CSVファイル名（デフォルト: `main2.0.csv`）
   - `seiteinen`: 制定年
   - `jichitai`: 自治体名
   - `kubun`: 区分
3. コードを実行